In [78]:
import pandas as pd
from pathlib import Path

In [2]:
ROOT_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim/"
DF_NAMES = (
    "CH4_df",
    "CO2_df",
    "HFC_df",
    "NF3_df",
    "PFC_df",
    "SF6_df",
    "N20_df",
)

CH4_df = pd.read_pickle(ROOT_DIR + DF_NAMES[0] + ".pkl")
CO2_df = pd.read_pickle(ROOT_DIR + DF_NAMES[1] + ".pkl")
HFC_df = pd.read_pickle(ROOT_DIR + DF_NAMES[2] + ".pkl")
NF3_df = pd.read_pickle(ROOT_DIR + DF_NAMES[3] + ".pkl")
PFC_df = pd.read_pickle(ROOT_DIR + DF_NAMES[4] + ".pkl")
SF6_df = pd.read_pickle(ROOT_DIR + DF_NAMES[5] + ".pkl")
N20_df = pd.read_pickle(ROOT_DIR + DF_NAMES[6] + ".pkl")

In [12]:
list_of_dfs = [CH4_df,
CO2_df,
HFC_df,
NF3_df,
PFC_df,
SF6_df,
N20_df]

In [39]:
DF_NAMES[0][:3]

'CH4'

In [32]:
test_df = pd.merge(N20_df, CO2_df, on = "Country or Area")

In [34]:
CO2_df.head()

,Country or Area,Year,Value
0,Australia,2018,415953.95
1,Australia,2017,415097.43
2,Australia,2016,411031.53
3,Australia,2015,401554.76
4,Australia,2014,394116.89


In [36]:
CO2_df.rename({"Country or Area":"Country", "Year":"Year", "Value":"CO2 Emissions"}, axis = 1, inplace = True)

In [40]:
for df, name in zip(list_of_dfs, DF_NAMES):
    df.rename({"Country or Area":"Country", "Year":"Year", "Value":f"{name[:3]} Emissions"}, axis = 1, inplace = True)

In [76]:
combined_df = pd.merge(CO2_df, NF3_df, how="outer")
combined_df = pd.merge(combined_df, CH4_df, how="outer")
combined_df = pd.merge(combined_df, HFC_df, how="outer")
combined_df = pd.merge(combined_df, PFC_df, how="outer")
combined_df = pd.merge(combined_df, SF6_df, how="outer")
combined_df = pd.merge(combined_df, N20_df, how="outer")


In [77]:
combined_df.head()

,Country,Year,CO2 Emissions,NF3 Emissions,CH4 Emissions,HFC Emissions,PFC Emissions,SF6 Emissions,N20 Emissions
0,Australia,2018,415953.95,NaN,109532.21,11982.08,236.00,228.53,20114.44
1,Australia,2017,415097.43,NaN,108170.70,11685.94,202.63,189.99,21265.59
2,Australia,2016,411031.53,NaN,105873.93,11979.46,224.92,186.31,19566.39
3,Australia,2015,401554.76,NaN,105368.80,11795.32,171.32,170.94,19557.63
4,Australia,2014,394116.89,NaN,105070.91,10778.85,192.54,155.17,20096.66


In [79]:
SAVE_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim"
file_path = Path(SAVE_DIR, "Combined_ghg_dataframe.pkl")

pd.to_pickle(combined_df, file_path)

In [80]:
ghg_2018  = combined_df[combined_df["Year"]==2018]
ghg_2018.head()

,Country,Year,CO2 Emissions,NF3 Emissions,CH4 Emissions,HFC Emissions,PFC Emissions,SF6 Emissions,N20 Emissions
0,Australia,2018,415953.95,NaN,109532.21,11982.08,236.00,228.53,20114.44
29,Austria,2018,66719.68,16.51,6438.63,1834.76,32.52,382.15,3526.08
58,Belarus,2018,61871.68,NaN,16097.62,NaN,NaN,3.46,14019.84
87,Belgium,2018,100207.84,0.65,7848.74,4469.84,131.32,95.08,5702.29
116,Bulgaria,2018,43551.60,NaN,6753.67,2252.53,0.01,17.99,5239.79


In [81]:
SAVE_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim"
file_path = Path(SAVE_DIR, "ghg_2018_dataframe.pkl")

pd.to_pickle(ghg_2018, file_path)